<a href="https://colab.research.google.com/github/sunflowerseed17/PythonDSProject/blob/main/Data_Preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# Importing dependencies
import os
import re
import time
!pip install praw
from datetime import datetime, timedelta
import praw
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer

### Scrape Reddit For Depression Posts (ONLY RUN ONCE)

In [ ]:
import os
import re
import time
from datetime import datetime, timedelta
import praw

# Configure Reddit API
reddit = praw.Reddit(
    client_id="ZaUY5qF9eLVVpD2OvHGEhg",
    client_secret="djHnirfkPnZUNI7XNs4dKUflOKjmtQ",
    user_agent="TextScraper by u/Jammberg",
    check_for_async=False
)

# List of related subreddits for specific scraping
subreddits = ["depression", "breastcancer"]

# Regex pattern for phrases indicating "I have been diagnosed with"
search_pattern = re.compile(
    r"(i\s+(was|am|have been|got|recently got|just got|was just|had been|found out i\s+was|"
    r"was diagnosed as having|diagnosed as suffering from|got diagnosed as having|received a diagnosis of|"
    r"was told i\s+have|was informed i\s+have)\s+.*)",
    re.IGNORECASE
)

# Function to create output folder
def create_folder(folder_name):
    if not os.path.exists(folder_name):
        os.makedirs(folder_name)

# Function to save a post to a file
def save_post(post, output_folder, filename_prefix="post"):
    filename = f"{filename_prefix}_{post.id}.txt"
    filepath = os.path.join(output_folder, filename)
    try:
        with open(filepath, "w", encoding="utf-8") as file:
            file.write(f"Subreddit: {post.subreddit.display_name}\n")
            file.write(f"Title: {post.title}\n")
            file.write(f"Author: {post.author}\n")
            file.write(f"Score: {post.score}\n")
            file.write(f"Created UTC: {datetime.utcfromtimestamp(post.created_utc)}\n")
            file.write(f"URL: {post.url}\n")
            file.write("\n")
            file.write(post.selftext)
        print(f"Saved post to {filepath}")
    except Exception as e:
        print(f"Error saving post {post.id}: {e}")

# Function to fetch posts by a specific user
def fetch_user_posts(username, reference_date, output_folder, max_posts=None):
    if not username:
        print("Author not available for this post.")
        return 0

    post_count = 0
    try:
        user = reddit.redditor(username)
        one_month_ago = reference_date - timedelta(days=30)
        for post in user.submissions.new(limit=None):
            if max_posts and post_count >= max_posts:
                break
            post_date = datetime.utcfromtimestamp(post.created_utc)
            if one_month_ago <= post_date <= reference_date and post.selftext.strip():
                save_post(post, output_folder, filename_prefix="user")
                post_count += 1
            elif post_date < one_month_ago:
                break
            time.sleep(2)  # Avoid hitting rate limits
    except Exception as e:
        print(f"Error fetching posts for user {username}: {e}")

    return post_count

# Function to fetch posts from a specific subreddit
def fetch_posts_from_subreddit(subreddit_name):
    print(f"\nFetching posts from r/{subreddit_name}...\n")
    subreddit = reddit.subreddit(subreddit_name)
    output_folder = f"reddit_scraped_posts/{subreddit_name}"
    create_folder(output_folder)

    try:
        for post in subreddit.new(limit=None):
            if post.selftext.strip() and re.search(search_pattern, post.selftext):
                reference_date = datetime.utcfromtimestamp(post.created_utc)
                save_post(post, output_folder, filename_prefix="subreddit")
                fetch_user_posts(post.author.name, reference_date, output_folder)
            time.sleep(2)  # Avoid hitting rate limits
    except Exception as e:
        print(f"Error fetching posts from r/{subreddit_name}: {e}")

# Function to fetch posts from r/all (new) within the last 2 months or until a maximum number of posts
def fetch_posts_from_all(max_posts=1100, max_retries=3):
    print("\nFetching posts from r/all (new)...\n")
    subreddit = reddit.subreddit("all")
    output_folder = "data/reddit_scraped_posts/standard"
    create_folder(output_folder)

    cutoff_date = datetime.utcnow() - timedelta(days=60)
    post_count = 0
    attempts = 0

    while attempts < max_retries:
        try:
            for post in subreddit.new(limit=None):
                if post_count >= max_posts:
                    break
                post_date = datetime.utcfromtimestamp(post.created_utc)
                if post_date < cutoff_date:
                    break
                if post.selftext.strip():
                    save_post(post, output_folder, filename_prefix="all")
                    post_count += 1

                    # Fetch additional posts from the same author
                    if post.author:
                        post_count += fetch_user_posts(post.author.name, post_date, output_folder, max_posts - post_count)

            break  # Exit the retry loop if successful
        except praw.exceptions.APIException as e:
            print(f"Rate limit exceeded: {e}. Retrying after a delay...")
            time.sleep(60)
            attempts += 1
        except Exception as e:
            print(f"Error fetching posts: {e}. Retrying...")
            time.sleep(30)
            attempts += 1

    print(f"\nTotal posts scraped: {post_count}")

# Main scraping logic
if __name__ == "__main__":
    for subreddit_name in subreddits:
        fetch_posts_from_subreddit(subreddit_name)

    fetch_posts_from_all()

    print("\nScraping complete! Text files saved in the respective folders.")


### Preprocessing.
#### The Code Below Does the Following:
1. Tokenization: Splitting text into individual tokens (words or punctuation marks).
2. Removing Noise: Cleaning the text by removing: URLs, Punctuation, and Stop words
3. Stemming: Reducing words to their root form (e.g., "running" -> "run").
4. Converting text to lowercase for consistency.

In [14]:
for category, paths in folders.items():
    input_folder = paths["input"]
    if os.path.exists(input_folder):
        print(f"\nContents of {input_folder}:")
        print(os.listdir(input_folder))
    else:
        print(f"\nInput folder does not exist: {input_folder}")



Contents of data\reddit_scraped_posts\depression:
[]

Contents of data\reddit_scraped_posts\breastcancer:
[]

Contents of data\reddit_scraped_posts\standard:
[]


In [13]:
# Download NLTK data files (only the first time)
nltk.download('punkt')
nltk.download('stopwords')

# Initialize tools
stop_words = set(stopwords.words('english'))
stemmer = PorterStemmer()

# Define folders for input and output
folders = {
    "depression": {
        "input": r"data\reddit_scraped_posts\depression",
        "output": r"data\preprocessed_posts\depression"
    },
    "breastcancer": {
        "input": r"data\reddit_scraped_posts\breastcancer",
        "output": r"data\preprocessed_posts\breastcancer"
    },
    "standard": {
        "input": r"data\reddit_scraped_posts\standard",
        "output": r"data\preprocessed_posts\standard"
    }
}

# Ensure both input and output folders exist
for category, paths in folders.items():
    os.makedirs(paths["input"], exist_ok=True)  # Create input folder if missing
    os.makedirs(paths["output"], exist_ok=True)  # Create output folder if missing

def preprocess_text(text):
    """
    Preprocess the given text:
    - Tokenize into words
    - Lowercase and remove non-alphanumeric tokens
    - Remove stop words
    - Apply stemming
    """
    try:
        # Tokenize text
        tokens = word_tokenize(text)
        
        # Remove URLs, punctuation, and stop words; lowercase the text
        tokens = [
            word.lower() for word in tokens 
            if word.isalnum() and word.lower() not in stop_words
        ]
        
        # Apply stemming
        tokens = [stemmer.stem(word) for word in tokens]
        
        # Join tokens back into a single string
        return " ".join(tokens)
    except Exception as e:
        print(f"Error preprocessing text: {e}")
        return ""

# Process each category
for category, paths in folders.items():
    input_folder = paths["input"]
    output_folder = paths["output"]
    
    if not os.path.exists(input_folder):
        print(f"Input folder does not exist: {input_folder}")
        continue

    print(f"\nProcessing {category} posts...")
    
    # Process each file in the input folder
    for filename in os.listdir(input_folder):
        if filename.endswith(".txt"):
            input_filepath = os.path.join(input_folder, filename)
            output_filepath = os.path.join(output_folder, filename)
            
            try:
                with open(input_filepath, "r", encoding="utf-8") as infile:
                    text = infile.read()
                
                # Extract post content (everything after the first header section)
                post_content = "\n".join(text.splitlines()[6:]).strip()
                
                # Skip files that are empty or contain '[removed]'
                if not post_content or '[removed]' in post_content.lower():
                    print(f"Skipping {filename} (empty or contains '[removed]')")
                    continue
                
                # Preprocess the text
                preprocessed_text = preprocess_text(post_content)
                
                # Save preprocessed text to a new file
                with open(output_filepath, "w", encoding="utf-8") as outfile:
                    outfile.write(preprocessed_text)
                
                print(f"Processed {filename} into {output_folder}")
            except Exception as e:
                print(f"Error processing file {filename}: {e}")

print("\nPreprocessing complete! Preprocessed files are saved in the respective output folders.")



Processing depression posts...

Processing breastcancer posts...

Processing standard posts...

Preprocessing complete! Preprocessed files are saved in the respective output folders.


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\jgber\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\jgber\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
